In [1]:
#Short Code to process the EXIF data from the images in the subfolder
import PIL.Image as pil
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import glob

In [2]:
# Test Print for one image
#img = pil.open('04_18_2020_1224/ZzZrBgs8kXhRRQPiaZBneQ.jpg')
#exif_data =img.getexif()
#print (exif_data)

In [3]:
class ImageMetaData(object):
    # Extract exif data from images. Process GPS data and get latitude
    # and longitudinal data
    exif_data = None
    image = None
    
    def __init__(self, img_path):
        self.image = Image.open(img_path)
        #print the exif data as a test
        #print(self.image._getexif())
        self.get_exif_data()
        super(ImageMetaData, self).__init__()
        
    def get_exif_data(self):
        # Returns the dictionary from the exif data of a PIL Image item.
        exif_data ={}
        info = self.image._getexif()
        if info:
            for tag,value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t,t)
                        gps_data[sub_decoded] = value[t]
                        
                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        self.exif_data = exif_data
        return exif_data

    def get_if_exist(self, data, key):
        if key in data:
            return data[key]
        return None
    
    def convert_to_degrees(self, value):
        # Convert GPS coordinates to float format
        d0 = value[0][0]
        d1 = value[0][1]
        d = float(d0) / float(d1)
        
        m0 = value[1][0]
        m1 = value[1][1]
        m = float(m0) / float(m1)
        
        s0 = value[2][0]
        s1 = value[2][1]
        s = float(s0) / float(s1)
        
        return d + (m / 60.0) + (s/3600.0)
    
    def get_lat_lng(self):
        # Return Lat and Long, if available, from the provided exif_data
        lat = None
        lng = None
        exif_data = self.get_exif_data()
        #print(exif_data)
        if "GPSInfo" in exif_data:
            gps_info = exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = self.convert_to_degrees(gps_latitude)
                if gps_latitude_ref != "N":                     
                    lat = 0 - lat
                lng = self.convert_to_degrees(gps_longitude)
                if gps_longitude_ref != "E":
                    lng = 0 - lng
        return lat, lng

In [6]:
path_name = '04_18_2020_1224/ZzY8jCoykYBve89GwMwTCA.jpg'
meta_data = ImageMetaData(path_name)
latlng = meta_data.get_lat_lng()
print("\n","GPS coordinates are: ",latlng)
exif_data = meta_data.get_exif_data()
#print("\n\n\n",exif_data)


 GPS coordinates are:  (40.76194047915887, -73.96213763927939)
